In [115]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [116]:
%autoreload 2

In [117]:
import requests
from dwre_tools.env import get_default_environment, get_default_project
from dwre_tools.sync import collect_cartridges, sync_command

In [118]:
CARTRIDGES = collect_cartridges("./test_cartridges/")
CARTRIDGES

./test_cartridges/


[('./test_cartridges/app_debugtest', 'app_debugtest')]

In [119]:
CLIENT_ID = "dwre-tools"
name, PROJECT = get_default_project()
name, ENV = get_default_environment(PROJECT)

In [120]:
AUTH = ENV['username'], ENV['password']
HEADERS = {
    "x-dw-client-id" : CLIENT_ID
}

In [138]:
ENV["verify"] = True
ENV["cert"] = None
sync_command(ENV, False, cartridge_location="test_cartridges/")

Syncing code version clavery on dev01-web-ktp.demandware.net
Extracting...
Successfully synced cartridges with dev01-web-ktp.demandware.net


In [122]:
BASE_URL = "https://{0}/s/-/dw/debugger/v1_0".format(ENV['server'])

In [123]:
session = requests.session()
session.auth = AUTH
session.headers = HEADERS

In [124]:
resp = session.delete(BASE_URL + "/client")

In [125]:
resp = session.post(BASE_URL + "/client")

In [126]:
resp = session.get(BASE_URL + "/breakpoints")
%resp resp

HTTP/1.1 200 OK
content-length: 12
accept-ranges: bytes
server: Apache
x-dw-request-base-id: WjsA-FiqAi2MAgAK
cache-control: max-age=0,no-cache,no-store,must-revalidate
date: Sun, 19 Feb 2017 20:38:05 GMT
content-type: application/json;charset=UTF-8

{
    "_v": "1.0"
}



In [127]:
BREAKPOINTS = {
    "breakpoints" : [
        {
          "line_number":32,
          "script_path":"/app_debugtest/cartridge/controllers/DebuggerTesting.js"
        }
    ]
}

In [128]:
resp = session.delete(BASE_URL + "/breakpoints")

resp = session.post(BASE_URL + "/breakpoints", json=BREAKPOINTS)
%resp resp

HTTP/1.1 200 OK
content-length: 126
accept-ranges: bytes
expires: Thu, 01 Dec 1994 16:00:00 GMT
vary: Accept-Encoding
server: Apache
x-dw-request-base-id: WjsG-FiqAjKMAgAK
pragma: no-cache
cache-control: no-cache,no-store,must-revalidate
date: Sun, 19 Feb 2017 20:38:10 GMT
content-type: application/json;charset=UTF-8

{
    "_v": "1.0", 
    "breakpoints": [
        {
            "line_number": 32, 
            "script_path": "/app_debugtest/cartridge/controllers/DebuggerTesting.js", 
            "id": 1
        }
    ]
}



In [130]:
resp = session.get(BASE_URL + "/threads")

threads = resp.json().get("script_threads")
if threads:
    thread_id = threads[0].get("id")

%resp resp

HTTP/1.1 200 OK
content-length: 360
accept-ranges: bytes
vary: Accept-Encoding
server: Apache
x-dw-request-base-id: Wjs6-FiqAkCMAgAK
cache-control: max-age=0,no-cache,no-store,must-revalidate
date: Sun, 19 Feb 2017 20:38:24 GMT
content-type: application/json;charset=UTF-8

{
    "_v": "1.0", 
    "script_threads": [
        {
            "status": "halted", 
            "id": 10, 
            "call_stack": [
                {
                    "index": 0, 
                    "location": {
                        "line_number": 32, 
                        "script_path": "/app_debugtest/cartridge/controllers/DebuggerTesting.js", 
                        "function_name": "start()"
                    }
                }, 
                {
                    "index": 1, 
                    "location": {
                        "line_number": 149, 
                        "script_path": "/app_storefront_controllers/cartridge/scripts/guard.js", 
                        "function_name"

In [109]:
resp = session.post(BASE_URL + "/threads/reset")
resp.status_code

204

In [131]:
resp = session.get(BASE_URL + "/threads/%s/frames/0/members" % thread_id)
%resp resp

HTTP/1.1 200 OK
content-length: 412
accept-ranges: bytes
vary: Accept-Encoding
server: Apache
x-dw-request-base-id: Wjs7-FiqAkaMAgAK
cache-control: max-age=0,no-cache,no-store,must-revalidate
date: Sun, 19 Feb 2017 20:38:30 GMT
content-type: application/json;charset=UTF-8

{
    "count": 4, 
    "_v": "1.0", 
    "total": 4, 
    "start": 0, 
    "object_members": [
        {
            "value": "org.mozilla.javascript.Arguments@1512db9", 
            "type": "com.demandware.core.script.internal.scriptobject.JSObjectImpl", 
            "name": "arguments", 
            "parent": ""
        }, 
        {
            "value": "foo", 
            "type": "String", 
            "name": "b", 
            "parent": ""
        }, 
        {
            "value": "[HashMap id=17807868]", 
            "type": "dw.util.HashMap", 
            "name": "myMap", 
            "parent": ""
        }, 
        {
            "value": "foo", 
            "type": "String", 
            "name": "t", 
     

In [136]:
resp = session.post(BASE_URL + "/threads/%s/resume" % thread_id)
resp.status_code

204

In [113]:
resp = session.get(BASE_URL + "/threads/%s/frames/0/eval" % thread_id, params={"expr" : "myMap.size()"})
%resp resp

HTTP/1.1 200 OK
content-length: 53
accept-ranges: bytes
server: Apache
x-dw-request-base-id: QDsa-Vip8uyMAgAK
cache-control: max-age=0,no-cache,no-store,must-revalidate
date: Sun, 19 Feb 2017 19:33:00 GMT
content-type: application/json;charset=UTF-8

{
    "_v": "1.0", 
    "expression": "myMap.size()", 
    "result": "2"
}



In [135]:
resp = session.get(BASE_URL + "/threads/%s/frames/0/members" % thread_id, params={"object_path" : "myMap"})
%resp resp

HTTP/1.1 200 OK
content-length: 180
accept-ranges: bytes
vary: Accept-Encoding
server: Apache
x-dw-request-base-id: HTt6-FiqAomMAgAK
cache-control: max-age=0,no-cache,no-store,must-revalidate
date: Sun, 19 Feb 2017 20:39:37 GMT
content-type: application/json;charset=UTF-8

{
    "count": 2, 
    "_v": "1.0", 
    "total": 2, 
    "start": 0, 
    "object_members": [
        {
            "value": "something", 
            "type": "String", 
            "name": "bob", 
            "parent": ""
        }, 
        {
            "value": "foo", 
            "type": "String", 
            "name": "yo", 
            "parent": ""
        }
    ]
}

